In [1]:
import pandas as pd
from urllib import request
import requests
import simplejson as json
import time
import os
import execjs

In [2]:
def get_user_space(mid):
    # 获取用户信息(空间)

    # url
    url = 'https://api.bilibili.com/x/space/acc/info?mid=%s'%(mid)
    # 请求网页
    response = request.urlopen(url)
    # 读取内容
    content = response.read()
    # 读取json
    dirt = json.loads(content)
    # 写入用户信息(空间)
    user_space = dirt['data']
    # 返回用户信息(空间)
    return user_space

In [3]:
def get_user_card(mid):
    # 获取用户信息(名片)

    # url
    url = 'https://api.bilibili.com/x/web-interface/card?mid=%s'%(mid)
    # 请求网页
    response = request.urlopen(url)
    # 读取内容
    content = response.read()
    # 读取json
    dirt = json.loads(content)
    # 写入用户信息(名片)
    user_card = dirt['data']
    # 返回用户信息(名片)
    return user_card

In [4]:
def get_user_followers(mid):
    # 获取用户粉丝明细

    user_followers = pd.DataFrame()
    # url
    url = 'https://api.bilibili.com/x/relation/followers?vmid=%s'%(mid)
    # 请求网页
    response = request.urlopen(url)
    # 读取内容
    content = response.read()
    # 读取json
    dirt = json.loads(content)
    # 写入用户粉丝明细
    user_followers = dirt['data']
    # 返回用户粉丝明细
    return user_followers

In [5]:
def get_user_followings(mid):
    # 获取用户关注明细

    user_followings = pd.DataFrame()
    # url
    url = 'https://api.bilibili.com/x/relation/followings?vmid=%s'%(mid)
    # 请求网页
    response = request.urlopen(url)
    # 读取内容
    content = response.read()
    # 读取json
    dirt = json.loads(content)
    # 写入用户关注明细
    user_followings = dirt['data']
    # 返回用户关注明细
    return user_followings

In [6]:
def user_info_output(cate_id, mid, sessdata):
    # 用户信息输出

    # 获取用户信息(空间)
    user_space = get_user_space(mid)
    # 获取用户信息(名片)
    user_card = get_user_card(mid)
    # 获取用户粉丝明细
    user_followers = get_user_followers(mid)
    # 获取用户关注明细
    user_followings = get_user_followings(mid)
    
    # 判断数据获取情况
    if (user_space and user_card and user_followers and user_followings):
        # 创建DataFrame
        # 用户信息(空间)
        output_user_space = pd.DataFrame([user_space])
        # 用户信息(名片)
        output_user_card = pd.DataFrame(data=user_card)
        # 用户粉丝明细
        output_user_followers = pd.DataFrame(data=user_followers)
        # 用户关注明细
        output_user_followings = pd.DataFrame(data=user_followings)

        # 获取当前目录
        current_path= os.getcwd()
        # 创建目录
        mkdir(current_path+'/data/user/{}'.format(mid))

        # 写入
        # 用户信息(空间)
        output_user_space.to_csv(current_path+'/data/user/{}/{}.csv'.format(mid, 'user_space'))
        # 用户信息(名片)
        output_user_card.to_csv(current_path+'/data/user/{}/{}.csv'.format(mid, 'user_card'))
        # 用户粉丝明细
        output_user_followers.to_csv(current_path+'/data/user/{}/{}.csv'.format(mid, 'user_followers'))
        # 用户关注明细
        output_user_followings.to_csv(current_path+'/data/user/{}/{}.csv'.format(mid, 'user_followings'))
        
        # 完成提示
        print(cate_id, '-', mid, ': 写入完成')
    else:
        # 失败提示
        print(cate_id, '-', mid, ': 写入失败')

In [7]:
def mkdir(path):
    # 判断路径是否存在
    isExists=os.path.exists(path)

    if not isExists:
        # 如果不存在则创建目录
        os.makedirs(path) 
        return True
    else:
        return False

In [8]:
def get_bvid_list(cate_id):
    # 获取bvid列表

    # 获取当前目录
    current_path= os.getcwd()
    # 读取CSV文件
    ranking = pd.read_csv(current_path+'/data/ranking/{}.csv'.format(cate_id))
    # 写入bvid列表
    bvid_list = pd.DataFrame(data=ranking, columns=['bvid'])

    return bvid_list

In [9]:
def bvID_to_CID(bvid):
    # bvID转CID(oid) 

    # url
    url = 'https://api.bilibili.com/x/player/pagelist?bvid=%s' %bvid
    # 请求网页
    response = request.urlopen(url)
    # 读取内容
    content = response.read()
    # 读取json
    dirt = json.loads(content)
    # 获取数量
    num = len(dirt['data'])
    # 创建列表
    cid_list = []
    # 循环
    for i in range(0, num):
        # 写入CID到列表
        cid_list.append(dirt['data'][i]['cid'])
    # 返回列表
    return cid_list

In [10]:
def get_user_id_list(cate_id, bvid, oid):
    # 获取user_id列表

    # 获取当前目录
    current_path= os.getcwd()
    # 读取CSV文件
    user_id = pd.read_csv(current_path+'/data/danmu/{}/{}/{}.csv'.format(cate_id, bvid, oid))
    # 写入user_id列表
    user_id_list = pd.DataFrame(data=user_id, columns=['user_id'])

    return user_id_list

In [11]:
def hash_crack(uidhash): 
    # 解密hash

    # 获取当前目录
    current_path= os.getcwd()
    # 读取js
    js = open(current_path+'/crc32-crack.js').read()
    # 执行js
    crack = execjs.compile(js)
    # 输出结果
    output = crack.call('crack_uidhash', uidhash)
    # 创建列表
    hash_crack = [output]
    # 返回结果
    return hash_crack

In [12]:
def danmu_to_user_list(cate_id, sessdata):
    # 获取用户id列表

    # 获取bvid列表
    bvid_list = get_bvid_list(cate_id)
    # 获取bvid数量
    bvid_num = bvid_list.shape[0]
    # 获取当前目录
    current_path= os.getcwd()

    # 创建DataFrame列表
    user_id_list = pd.DataFrame()
    user_id_list_crack = pd.DataFrame()
    
    # 判断路径是否存在
    isExists=os.path.exists(current_path+'/data/user/{}_original.csv'.format(cate_id))
    if isExists:
        # 读取未解密列表
        user_id_list = pd.read_csv(current_path+'/data/user/{}_original.csv'.format(cate_id))
    else:
        # 开始获取用户列表提示
        print(cate_id, ': 正在获取用户列表', '\n')
        # 循环
        for i in range(0, bvid_num):
            # 获取bvid
            bvid = bvid_list.iat[i, 0]

            # 判断路径是否存在
            isExists=os.path.exists(current_path+'/data/danmu/{}/{}'.format(cate_id, bvid))
            if isExists:
                try:
                    # 获取oid列表
                    oid_list = bvID_to_CID(bvid)
                    # 获取oid数量
                    oid_num = len(oid_list)
                    # 等待3秒
                    time.sleep(3)

                    # 循环
                    for o in range(0, oid_num):
                        # 获取user_id列表
                        user_id_list = user_id_list.append(get_user_id_list(cate_id, bvid, oid_list[o]))
                        # 去重
                        user_id_list.drop_duplicates(inplace=True)
                        # 获取user_id数量
                        user_id_num = user_id_list.shape[0]
                        # 进度提示
                        print('user_id数量: ', user_id_num, '\n')
                except Exception as e:
                    # 获取失败
                    print(bvid, ': 获取失败')
                    print('错误: ', e, '\n')
                    # 等待3秒
                    time.sleep(3)
        # 写入未解密列表
        user_id_list.to_csv(current_path+'/data/user/{}_original.csv'.format(cate_id))
    
    # 读取未解密列表
    user_id_list = pd.read_csv(current_path+'/data/user/{}_original.csv'.format(cate_id))
    # 循环
    for u in range(0, 5000):
        # 获取user_id
        user_id = user_id_list.iat[u, 1]
        # 解密并写入user_id_list_crack列表
        user_id_list_crack = user_id_list_crack.append(hash_crack(user_id)[0])
        # 去重
        user_id_list_crack.drop_duplicates(inplace=True)
        # 获取user_id数量
        user_id_list_crack_num = user_id_list_crack.shape[0]
        # 进度提示
        print('user_id解密数量: ', user_id_list_crack_num, '\n')               
    # 写入用户id列表
    user_id_list_crack.to_csv(current_path+'/data/user/{}.csv'.format(cate_id))
    
    # 读取用户id列表
    user_id_list_crack = pd.read_csv(current_path+'/data/user/{}.csv'.format(cate_id))
    # 返回用户id列表
    return user_id_list_crack

In [13]:
def user_list_to_user_info(cate_id, sessdata):
    # 获取用户信息

    # 获取当前目录
    current_path= os.getcwd()
    # 判断路径是否存在
    isExists=os.path.exists(current_path+'/data/user/{}.csv'.format(cate_id))
    if isExists:
        # 读取用户id列表
        user_id_list = pd.read_csv(current_path+'/data/user/{}.csv'.format(cate_id))
    else:
        # 获取用户id列表
        user_id_list = pd.DataFrame(data=danmu_to_user_list(cate_id, sessdata))

    # 完成获取用户id列表提示
    print(cate_id, ': 获取用户列表完成', '\n')

    # 获取用户id数量
    user_id_list_num = user_id_list.shape[0]
    # 循环
    for i in range(0, user_id_list_num):
        # 获取用户id
        mid = user_id_list.iat[i, 1]
        # 判断路径是否存在
        isExists=os.path.exists(current_path+'/data/user/{}'.format(mid))
        if not isExists:
            try:
                # 获取用户信息
                user_info_output(cate_id, mid, sessdata)
                # 空行
                print('\n')
                # 等待5秒
                time.sleep(5)
            except Exception as e:
                # 获取失败
                print(cate_id, '-', mid, ': 获取失败')
                print('错误: ', e, '\n')
                # 等待5秒
                time.sleep(5)
    # 全部完成提示
    print(cate_id, ': 用户信息写入完成')

In [ ]:
# 分区号, SESSDATA
user_list_to_user_info('76', 'de6539ff%2C1622284640%2C83390*b1')

76 : 获取用户列表完成 

76 - 980250362 : 获取失败
错误:  'data' 

76 - 873075070 : 获取失败
错误:  'data' 

76 - 919718231 : 获取失败
错误:  'data' 

76 - 805023861 : 获取失败
错误:  'data' 

76 - 909965591 : 获取失败
错误:  'data' 

76 - 988790917 : 获取失败
错误:  'data' 

76 - 963801485 : 获取失败
错误:  'data' 

76 - 805526560 : 获取失败
错误:  'data' 

76 - 926955285 : 获取失败
错误:  'data' 

76 - 905254482 : 获取失败
错误:  'data' 

76 - 817150176 : 获取失败
错误:  'data' 

76 - 905994284 : 获取失败
错误:  'data' 

76 - 805756978 : 获取失败
错误:  'data' 

76 - 855443184 : 获取失败
错误:  'data' 

76 - 814021987 : 获取失败
错误:  'data' 

76 - 813137897 : 获取失败
错误:  'data' 

76 - 928150357 : 获取失败
错误:  'data' 

76 - 995950693 : 获取失败
错误:  'data' 

76 - 905995580 : 获取失败
错误:  'data' 

76 - 902989483 : 获取失败
错误:  'data' 

76 - 912654595 : 获取失败
错误:  'data' 

76 - 811549670 : 获取失败
错误:  'data' 

76 - 835437297 : 获取失败
错误:  'data' 

76 - 998806535 : 获取失败
错误:  'data' 

76 - 911440046 : 获取失败
错误:  'data' 

76 - 921609354 : 获取失败
错误:  'data' 

76 - 928469457 : 获取失败
错误:  'data' 

76 - 9108857

76 - 733772783 : 获取失败
错误:  'data' 

76 - 399845749 : 写入完成


76 - 911978618 : 获取失败
错误:  'data' 

76 - 517431846 : 写入完成


76 - 76681150 : 写入完成


76 - 393620799 : 写入完成


76 - 390794626 : 写入完成


76 - 382157855 : 写入完成


76 - 179455171 : 写入完成


76 - 375320576 : 写入完成


76 - 123160398 : 写入完成


76 - 40020352 : 写入完成


76 - 979668615 : 获取失败
错误:  'data' 

76 - 35325263 : 写入完成


76 - 384450675 : 写入完成


76 - 818187935 : 获取失败
错误:  'data' 

76 - 917388591 : 获取失败
错误:  'data' 

76 - 479979223 : 写入完成


76 - 519354398 : 写入完成


76 - 182531867 : 写入完成


76 - 379210410 : 写入完成


76 - 953995000 : 获取失败
错误:  'data' 

76 - 449510222 : 写入完成


76 - 228912731 : 写入完成


76 - 802297321 : 获取失败
错误:  'data' 

76 - 819472194 : 获取失败
错误:  'data' 

76 - 352431513 : 写入完成


76 - 406928333 : 写入完成


76 - 13896939 : 写入完成


76 - 454590139 : 写入完成


76 - 12697092 : 写入完成


76 - 28201073 : 写入完成


76 - 320929391 : 写入完成


76 - 399574465 : 写入完成


76 - 911449534 : 获取失败
错误:  'data' 

76 - 6773948 : 写入完成


76 - 97304459 : 写入完成


76 - 6236108 